___

# Machine Learning in Geosciences ] 
Department of Applied Geoinformatics and Carthography, Charles University

Lukas Brodsky lukas.brodsky@natur.cuni.cz


## Deep Learning, Convolutional Neural Networks


This notebook introduces the Deep Learning with PyTorch https://pytorch.org: 


* PyTorch CNN
    


### PyTorch installation

`pip install torch`
`pip install torchvision`

PyTorch includes a package called torchvision which is used to load and prepare the dataset. It includes two basic functions namely Dataset and DataLoader which helps in transformation and loading of dataset.

# Setup

In [ ]:
# Common imports
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch imports 
import torch
import torch.nn as nn # Neural Network class
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable


# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Project dir
PROJECT_DIR = "./"
if os.path.isdir(PROJECT_DIR):
    print('Ok continue.')
else:
    print('Nok, set correct path to your project directory!')

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

### PyTorch CNN

#### Main steps 
1. Load dataset
2. Make dataset iterable
3. Create model class
4. Instantiate model class
5. Instantiate loss class
6. Instantiate optimizer class
7. Train model

In [ ]:
""" Step 1: Load MNIST Dataset """

# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

In [ ]:
# from torchvision.datasets import MNIST
# transform = transforms.Compose([transforms.ToTensor()])
# train_data = MNIST(root = './MNIST', train=True, download=False, transform=transform)
# val_data = MNIST(root = './MNIST', train=False, download=False, transform=transform)

In [ ]:
""" Step 1: Load MNIST Dataset """

train_dataset = dsets.MNIST(root='./data', train = True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
print(train_dataset.data.size())
print(train_dataset.targets.size())

In [ ]:
print(test_dataset.data.size())
print(test_dataset.targets.size())

In [ ]:
train_dataset_array = next(iter(train_dataset))[0].numpy()
# train_dataset_array.shape
plt.imshow(train_dataset_array[0,:,:], cmap='gray', interpolation='none')

In [ ]:
next(iter(train_dataset.train_labels)).numpy()

In [ ]:
""" Step 2: Make it iterable """

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

In [ ]:
print(num_epochs)
print(len(train_dataset))
print(batch_size)

In [ ]:
""" Step 3: Create Model Class """

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(16 * 7 * 7, 10)


    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2
        out = self.maxpool2 ( out )

        # Resize
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)

        return out


In [ ]:
""" Step 4: Instantiate Model Class """

model = CNNModel()

In [ ]:
# PARAMETERS 
# print(model.parameters())
# print(len(list(model.parameters())))

# Convolution 1: 16 kernels
# print(list(model.parameters())[0].size())

# Convolution 1 Bias: 16 kernels
# print(list(model.parameters())[1].size())

# Convolution 2: 16 kernels
# print(list(model.parameters())[2].size())

# Convolution 2 Bias: 16 kernels
# print(list(model.parameters())[3].size())

# Fully connected layer
# print(list(model.parameters())[4].size())

# Fully connected layer Bias
# print(list(model.parameters())[5].size())

In [ ]:
""" Step 5: Initiate Loss Class """

# CNN: Cross Entropy Loss
criterion = nn.CrossEntropyLoss()

In [ ]:
""" Step 6: Instantiate Optimizer Class """

learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
""" Step 7: Train Model """

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # load images as Variables
        images = Variable(images) # X
        labels = Variable(labels) # y 

        # clear gradients w.r.t. parameters
        optimizer.zero_grad()
        # forward pass to get output 
        outputs = model(images)
        # calculate cross entropy loss
        loss = criterion(outputs, labels)
        # getting gradients w.r.t. parameters
        loss.backward()
        # updating parameters
        optimizer.step()
        iter += 1

        if iter % 100 == 0:
            # Calculate accuracy
            correct = 0
            total = 0
            # iterate through test data set
            for images, labels in test_loader:
                # load images to a Torch Variable
                images = Variable(images)

                # forward pass only to get output
                outputs = model(images)
                # get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                # total number of labels
                total += labels.size(0)
                # total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # print loss
            # loss.data[0] X loss.item()
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))


In [ ]:
accuracy

In [ ]:
# Task 1: plot graph of loss and accuracy with iterations

In [ ]:
# Task 2: implement early stopping, if needed